In [ ]:
%load_ext autoreload
%autoreload 2

# Setup Agents


In [ ]:
# Lets you run asyncio run in a notebook
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os

api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from meadow.cache.duckdb import DuckDBCache
from meadow.client.api.openai import OpenAIClient
from meadow.client.client import Client
from meadow.client.schema import LLMConfig
from meadow.database.connector.sqlite import SQLiteConnector
from meadow.database.database import Database

db_ambig_path = "/home/lorr1/projects/code/meadow/examples/data/database_sqlite/sales_ambiguous_joins_example/sales_ambiguous_joins_example.sqlite"
connector = SQLiteConnector(db_ambig_path)
database = Database(connector)

# LLM cache for making reruns faster (optional)
cache = DuckDBCache("test_cache.duckdb")
# Model
api_client = OpenAIClient(api_key=api_key)
model = "gpt-4o"

client = Client(
    cache=cache,
    api_client=api_client,
    model=model,
)
llm_config = LLMConfig(
    max_tokens=1000,
    temperature=0.0,
)

In [ ]:
# Build you custom agent to work with text2SQL. Suppose we want a result explain that explains the result to the user.
from meadow.agent.data_agents.basic import BasicAgent

explainer_agent = BasicAgent(
    name="ExplainSQLResult",
    description="This agent explains the result of a SQL query in text. Use it after a SQL generation agent.",
    system_prompt="""Given the schema
{schema}

Please explain the SQL query provided. If no SQL is provided in the conversation, use the largest SQL view in the schema.""",
    client=client,
    llm_config=llm_config,
    database=database,
)

# Start a Conversation


In [ ]:
import asyncio

from meadow.agent.controller import ControllerAgent
from meadow.agent.data_agents.text2sql import SQLGeneratorAgent
from meadow.agent.planner import PlannerAgent
from meadow.agent.user import UserAgent

text2sql = SQLGeneratorAgent(
    client=client,
    llm_config=llm_config,
    database=database,
)
planner = PlannerAgent(
    available_agents=[
        explainer_agent,  # Add the agent to the planner
        text2sql,
    ],
    client=client,
    llm_config=llm_config,
    database=database,
)
user = UserAgent(name="User", silent=False)
controller = ControllerAgent(
    supervisor=user, planner=planner, database=database, silent=True
)

question = "What is the average total revenue for customers with different marital statuses and education levels?"
final_response = asyncio.run(controller.initiate_chat(input=question))